# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [ ]:
def priest(sex, age, respiratory_rate, o2_sat, heart_rate, bp_sys, temp_C, alertness, o2_inspired, performance_status):
    '''This calculates COVID-19 risk score for a patient and returns the probability of adverse effects.It uses the algorithm from MDCalc website.
       For a person who takes their vitals parameters into consideration, the first part of this function will calculate COVID-19 severity and
       the risk score can help predict the patient's probability of adverse effects in the next 30 days.


    >>> priest('MALE',40,20,0.99,135,115,39.2,'ALERT','AIR','unrestricted normal activity')
    0.15
    >>> priest('FEMALE',18,22,0.96,105,101,38.1,'ALERT','AIR','unrestricted normal activity')
    0.09
    >>> priest('MalE', 65, 24, 0.87, 90, 99, 36.5, 'ConFusEd oR nOt AleRt', 'air', 'limited strenuous activity, can do light activity')
    0.47
    >>> priest('Female',18,10,0.96,90,230,38.1,'ALERT','AIR','limited self-care')
    0.22
    '''


    sex_score = 0
    age_score = 0
    respiratory_rate_score = 0
    o2_sat_score = 0
    heart_rate_score = 0
    bp_sys_score = 0
    temp_C_score = 0
    alertness_score = 0
    o2_inspired_score = 0
    performance_status_score = 0

    if sex.lower() == 'male':
      sex_score = 1
    elif sex.lower() == 'female':
      sex_score = 0

    if 16 <= age <= 49:
        age_score = 0
    elif 50 <= age <= 65:
        age_score = 2
    elif 66 <= age <= 80:
        age_score = 3
    elif age > 80:
        age_score = 4

    if respiratory_rate < 9:
        respiratory_rate_score = 3
    elif 9 <= respiratory_rate <= 11:
        respiratory_rate_score = 1
    elif 12 <= respiratory_rate <= 20:
        respiratory_rate_score = 0
    elif 21 <= respiratory_rate <= 24:
        respiratory_rate_score = 2
    elif respiratory_rate > 24:
        respiratory_rate_score = 3

    if o2_sat > 0.95:
        o2_sat_score = 0
    elif 0.94 <= o2_sat <= 0.95:
        o2_sat_score = 1
    elif 0.92 <= o2_sat <= 0.93:
        o2_sat_score = 2
    else:
        o2_sat_score = 3

    if heart_rate < 41:
        heart_rate_score = 3
    elif 41 <= heart_rate <= 50:
        heart_rate_score = 1
    elif 51 <= heart_rate <= 90:
        heart_rate_score = 0
    elif 91 <= heart_rate <= 110:
        heart_rate_score = 1
    elif 111 <= heart_rate <= 130:
        heart_rate_score = 2
    else:
        heart_rate_score = 3

    if bp_sys < 91:
        bp_sys_score = 3
    elif 91 <= bp_sys <= 100:
        bp_sys_score = 2
    elif 101 <= bp_sys <= 110:
        bp_sys_score = 1
    elif 111 <= bp_sys <= 219:
        bp_sys_score = 0
    else:
        bp_sys_score = 3

    if temp_C < 35.1:
        temp_C_score = 3
    elif 35.1 <= temp_C <= 36.0:
        temp_C_score = 1
    elif 36.1 <= temp_C <= 38.0:
        temp_C_score = 0
    elif 38.1 <= temp_C <= 39.0:
        temp_C_score = 1
    else:
        temp_C_score = 2

    if alertness.lower() == 'confused or not alert':
        alertness_score = 3

    if o2_inspired.lower() == 'supplemental oxygen':
        o2_inspired_score = 2

    if performance_status.lower() == 'limited strenuous activity, can do light activity':
        performance_status_score = 1
    elif performance_status.lower() == 'limited activity, can self-care':
        performance_status_score = 2
    elif performance_status.lower() == 'limited self-care':
        performance_status_score = 3
    elif performance_status.lower() == 'bed/chair bound, no self-care':
        performance_status_score = 4

    risk_score = sex_score + age_score + respiratory_rate_score + o2_sat_score + heart_rate_score + bp_sys_score + temp_C_score + alertness_score + o2_inspired_score + performance_status_score

    if risk_score >= 0 and risk_score <= 1:
        risk = 0.01
    elif risk_score >= 2 and risk_score <= 3:
        risk = 0.02
    elif risk_score == 4:
        risk = 0.03
    elif risk_score == 5:
        risk = 0.09
    elif risk_score == 6:
        risk = 0.15
    elif risk_score == 7:
        risk = 0.18
    elif risk_score == 8:
        risk = 0.22
    elif risk_score == 9:
        risk = 0.26
    elif risk_score == 10:
        risk = 0.29
    elif risk_score == 11:
        risk = 0.34
    elif risk_score == 12:
        risk = 0.38
    elif risk_score == 13:
        risk = 0.46
    elif risk_score == 14:
        risk = 0.47
    elif risk_score == 15:
        risk = 0.49
    elif risk_score == 16:
        risk = 0.55
    elif risk_score >= 17 and risk_score <= 25:
        risk = 0.59
    else:
        risk = 0.99

    return risk

In [9]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('MALE',40,20,0.99,135,115,39.2,'ALERT','AIR','unrestricted normal activity')
Expecting:
    0.15
ok
Trying:
    priest('FEMALE',18,22,0.96,105,101,38.1,'ALERT','AIR','unrestricted normal activity')
Expecting:
    0.09
ok
Trying:
    priest('MalE', 65, 24, 0.87, 90, 99, 36.5, 'ConFusEd oR nOt AleRt', 'air', 'limited strenuous activity, can do light activity')
Expecting:
    0.47
ok
Trying:
    priest('Female',18,10,0.96,90,230,38.1,'ALERT','AIR','limited self-care')
Expecting:
    0.22
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [ ]:
import requests
import json
def find_hospital(age, sex, risk):
    ''' In this function, a patient's risk score, age and sex are calculated to determine which location where the person can be treated.
        If they're using the REST web service in an emergency.

    >>> find_hospital(52, 'female', 0.15)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(66, 'male', 0.4)
    'Wesley Woods Geriatric Hospital'
    >>> find_hospital(30, 'female', 0.51)
    'Emory Dunwoody Medical Center'
    >>> find_hospital(25, 'male', 0.10)
    'Southwest Hospital and Medical Center'
    '''
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    response = requests.get(url)
    data = response.json()
    hospital_name = data['hospital']
    return (hospital_name)

In [ ]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(52, 'female', 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(66, 'male', 0.4)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(30, 'female', 0.51)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(25, 'male', 0.10)
Expecting:
    'Southwest Hospital and Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [ ]:
def get_hospital_address(hospital_name):
  ''' We should better find the address of the hospital when we found out which one to go to in an emergency.
      Look up the hospital name in a JSON file containing hospital names and addresses and Return the address string for this hospital name.

    >>> get_hospital_address('ST MARYS HEALTH CARE SYSTEM')
    '1230 BAXTER STREET'
    >>> get_hospital_address('EMORY DUNWOODY MEDICAL CENTER')
    '4500 NORTH SHALLOWFORD ROAD'
    >>> get_hospital_address('HILLSIDE HOSPITAL')
    '690 COURTENAY DRIVE NE'
    >>> get_hospital_address('PRIDE MEDICAL')
    '3280 HOWELL MILL ROAD NW'
    '''

  with open('ga_hospitals.json') as f:
    hospitals = json.load(f)
    hospital_name = hospital_name.upper()
    return hospitals[hospital_name]['ADDRESS']

In [ ]:
import doctest
doctest.run_docstring_examples(get_hospital_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_hospital_address('ST MARYS HEALTH CARE SYSTEM')
Expecting:
    '1230 BAXTER STREET'
ok
Trying:
    get_hospital_address('EMORY DUNWOODY MEDICAL CENTER')
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok
Trying:
    get_hospital_address('HILLSIDE HOSPITAL')
Expecting:
    '690 COURTENAY DRIVE NE'
ok
Trying:
    get_hospital_address('PRIDE MEDICAL')
Expecting:
    '3280 HOWELL MILL ROAD NW'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [ ]:
def process_people(file_name):
   """ A CSV file is used to process patient's data. This calculates the risk score, finds a hospital for each patient and gets their address.
   The results are stored according to the patient's number in a dictionary. """
   results = {}
   with open(file_name) as csv_file:
        lines = csv_file.readlines()


        for line in lines[1:]:
            try:

                patient_details = line.strip().split("|")

                patient_number,sex, age, respiratory_rate, o2_sat, heart_rate, bp_sys, temp_C, alertness, o2_inspired, performance_status = patient_details



                risk_score = priest(sex, int(age), int(respiratory_rate), float(o2_sat), int(heart_rate), int(bp_sys), float(temp_C), alertness, o2_inspired, performance_status)

                hospital_name = find_hospital(int(age), sex, risk_score)

                hospital_address = get_hospital_address(hospital_name)



                results[patient_number] = [sex, int(age), float(respiratory_rate), float(o2_sat), float(heart_rate), float(bp_sys), float(temp_C), alertness, o2_inspired, performance_status
                                           , risk_score, hospital_name, hospital_address]

            except Exception as e:
                print("Error processing patient:", e)
   return results

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [ ]:
import json
json_file = "people_results.json"
with open(json_file) as file:
    json_data = json.loads(file.read())
    peoples_results = process_people('people.psv')
    if json_data == peoples_results:
        print("Both are Same")
    else:
      print("Both are different")




Both are Same


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---